# Datasets Ensenyament maig 2022

En el marc de la [Setmana del Govern Obert](https://educacio.gencat.cat/ca/departament/transparencia/govern-obert/) es varen publicar dos datasets interessants, ["Plantilles del personal docent dels centres i serveis educatius i forma d'ocupació"](https://analisi.transparenciacatalunya.cat/Educaci-/Plantilles-del-personal-docent-dels-centres-i-serv/9zqj-j7ei) i "Concerts educatius: Unitat, alumnes, dotació de plantilla i despesa".

Aquest bloc conté unes visualitzacions sobre les plantilles de profes d'informàtica de Girona basades en les dades del primer dels conjunts de dades esmentats.

L'objectiu d'aquest bloc és tenir una foto de la situació actual del professorat d'informàtica a les comarques gironines així com a servir d'inspiració per a altres visualitzacions i usus que se li vulguin donar a aquestes dades o altres dades obertes. 

Sigueu lliures de fer PR o fork [del repo a GitHub](https://github.com/ctrl-alt-d/dataset_ensenyament_2022). Has trobat cap error? Tens alguna idea per millorar aquestes gràfiques? Sisplau, [obra issue](https://github.com/ctrl-alt-d/dataset_ensenyament_2022/issues)

## Càrrega de les dades

Carreguem el dataset de plantilles i ens quedem amb els informàtics gironins.

In [85]:
import pandas as pd

url="https://analisi.transparenciacatalunya.cat/api/views/9zqj-j7ei/rows.csv?accessType=DOWNLOAD&sorting=true"

df = pd.read_csv(url, on_bad_lines='skip')

# El codi centre és un string:
df.CODI_CENTRE= df.CODI_CENTRE.astype(str)

# Ens quedem amb els informàtics gironins:
df_girona = df.loc[df.COD_LLOC.isin( ['507','627'] ) & df.CODI_CENTRE.str.startswith('17') ]


## Dotació curs 2021-2022

Dotació per centre de professorat de les especialitats d'informàtica

In [86]:
import altair as alt

data = df_girona[df_girona.Curs=="2021-2022"]

bars = alt.Chart().mark_bar().encode(
    alt.X("sum(TOT_DOT):Q",title="Dotació", stack='zero'),
    alt.Y("NOM_MUNICIPI:N", title=""),
    alt.Color("COD_LLOC", title="Especialitat"),
)

text_detall = alt.Chart().mark_text(dx=-13, color='white').encode(
    alt.X("sum(TOT_DOT):Q", stack='zero'),
    alt.Y("NOM_MUNICIPI:N", title=""),
    detail='COD_LLOC:N',
    text=alt.Text('sum(TOT_DOT):Q')
)

alt.layer(
        bars, text_detall, data=data
    ).interactive()

alt.LayerChart(...)

## Evolució de la Plantilla per centre

Nombre de professors per especialitat a cada centre.

In [87]:
import altair as alt

data = df_girona

base = alt.Chart(data).encode(
    alt.Y("sum(TOT_DOT)",title="Dotació", stack='zero'),
    alt.X("Curs:N", title=""),
)

bars = base.mark_bar().encode(
    alt.Color("COD_LLOC", title="Especialitat"),
)

text_detall = base.mark_text(dy=10, color='white').encode(
    detail='COD_LLOC:N',
    text=alt.Text('sum(TOT_DOT):Q')
)

alt.layer(
        bars, text_detall
    ).facet(
        facet=alt.Column('NOM_MUNICIPI', title=""),
        columns=5
    ).interactive()


alt.FacetChart(...)

## Destinació del professorat als centres

Destinació del professorat als centres.

* Funcionari amb destinació definitiva al centre:
  * DD: Destinació definitiva
* Funcionari sense destinació definitiva al centre:
  * CS: Comissió de serveis
  * PP: funcionari amb destinació provisional
  * PS: funcionari que ha perdut la definitiva per supressió de la plaça al centre on estava i està provisionalment a aquesta destinació
* Interinatge, personal de reforç i no fix:
  * IN:	Interinatge
  * NF:	no fix
  * PG: personal reforç pel tema del COVID
  * RF: Valor de reforços

### Destinació curs 2021-2022

Destinació del professorat als centres curs 2021-2022 a les comarques gironines.

In [88]:
import altair as alt

# Tipus i colors de les destinacions
destinacions = ["DD", "CS", "PP", "PS", "IN", "NF", "PG", "RF"]
colors = ['#229954', '#2E86C1', '#2471A3', '#884EA0', '#D35400', '#CA6F1E', '#D68910', '#F39C12', '#D4AC0D']

# Taula curs, municipi, destinació, Dotació
data = pd.melt(df_girona[df_girona.Curs=="2021-2022"], id_vars=[], value_vars=destinacions, var_name="Destinació", value_name="Dotació").dropna()

# Camp addicional per a ordenació manual
data['order'] = data['Destinació'].replace(
    {val: i for i, val in enumerate(destinacions)}
)

base = alt.Chart(data).encode(
    theta=alt.Theta("sum(Dotació):Q", stack=True),
    radius=alt.Radius("sum(Dotació)", scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)),
    color=alt.Color("Destinació", title="Vinculació al centre", scale=alt.Scale(domain=destinacions, range=colors)),
)

c1 = base.mark_arc(innerRadius=20, stroke="#fff")

c2 = base.mark_text(radiusOffset=10).encode(text="sum(Dotació):Q")

alt.layer(c1, c2)

alt.LayerChart(...)

In [89]:
import altair as alt

# Taula curs, municipi, destinació, Dotació
data = pd.melt(df_girona, id_vars=["Curs", "NOM_MUNICIPI"], value_vars=destinacions, var_name="Destinació", value_name="Dotació").dropna()

# Camp addicional per a ordenació manual
data['order'] = data['Destinació'].replace(
    {val: i for i, val in enumerate(destinacions)}
)

base = base = alt.Chart(data).encode(
    alt.Y("sum(Dotació)",title="Dotació", stack='zero' ),
    alt.X("Curs:N", title=""),
    alt.Order("order", sort="descending" ),
)

bars = base.mark_bar().encode(
    alt.Color("Destinació", title="Vinculació al centre", scale=alt.Scale(domain=destinacions, range=colors)),
)

text_detall = base.mark_text(dy=10, color='white').encode(
    detail='Destinació:N',
    text=alt.Text('sum(Dotació):Q'),
)

alt.layer(
        bars, text_detall
    ).facet(
        facet=alt.Column('NOM_MUNICIPI', title=""),
        columns=5
    ).interactive()

alt.FacetChart(...)

## Places ofertades i assignacions

Completo la visualització amb les dades del dataset [Estadística de l'assignació de places en el procés de la preinscripció en els ensenyaments post-obligatoris](https://analisi.transparenciacatalunya.cat/Educaci-/Estad-stica-de-l-assignaci-de-places-en-el-proc-s-/wjaf-jxmt)

In [90]:
# Busco i preparo les dades, em quedo amb informàtica 2021-2022

import pandas as pd

url = "https://analisi.transparenciacatalunya.cat/api/views/wjaf-jxmt/rows.csv?accessType=DOWNLOAD&sorting=true"

df_oferta = pd.read_csv(url, on_bad_lines='skip')

# El codi centre és un string:
df_oferta["Codi centre"]= df_oferta["Codi centre"].astype(str)

estudis = [
    "Sistemes microinformàtics i xarxes", 
    "Desenvolupament d'aplicacions multiplataforma", 
    "Desenvolupament d'aplicacions web", 
    "Administració de sistemes informàtics en xarxa", 
    ]

codis = ["SMX", "DAM", "DAW", "ASIX"]

# Ens quedem amb els informàtics gironins:
df_girona = df_oferta.loc[df_oferta["Codi centre"].str.startswith('17') & df_oferta["Nom ensenyament"].isin(estudis)].copy()

df_girona['Cicle'] = df_girona['Nom ensenyament'].replace(
    {val: codis[i] for i, val in enumerate(estudis)}
)

df_girona_2122 = df_girona[df_girona.Curs == "2021/2022"]

### Visió global de places ofertades i places assignades

In [91]:
import altair as alt

torns = list(df_girona_2122["Torn"].drop_duplicates())

data = df_girona_2122

oferta = alt.Chart(data).mark_bar().encode(
    alt.X("sum(Nombre places):Q",title=f"Oferta", stack='zero'),
    alt.Color("Cicle:N"),
    alt.Y('Nom municipi', title="") 
).interactive()

assignacions = alt.Chart(data).mark_bar().encode(
    alt.X("sum(Assignacions):Q",title=f"Assignacions", stack='zero'),
    alt.Color("Cicle:N"),
    alt.Y('Nom municipi', title="") 
).interactive()

(oferta | assignacions)


alt.HConcatChart(...)

### Oferta i assignacions per torn i cicle

Les barres representen l'oferta de places, la marca verda representa les assignacions.

In [92]:
import altair as alt

torns = list(df_girona_2122["Torn"].drop_duplicates())

data = df_girona_2122

base = alt.Chart(data).encode(
    alt.X("Cicle:N", title=""),
)

places = base.mark_bar().encode(
    alt.Y("sum(Nombre places):Q",title=f"Oferta", stack='zero'),
    alt.Color("Torn:N", title="Torn", scale=alt.Scale(scheme='dark2')),    
)

assignacions = base.mark_tick(
    color='green',
    thickness=2,
).encode(
    alt.Y("sum(Assignacions):Q",title=f"Oferta", stack='zero'),
)

alt.layer(
        places, assignacions
).facet(
        facet=alt.Column('Nom municipi', title="") 
).interactive()





alt.FacetChart(...)